## Data Wrangling - Open Street Map
### Introdcution

For my Open Street Map project I selected a section of map that I am familiar with.  This is a subset of Salt Lake City, Utah.  I selected the area contained within the sourthern loop of Interstate 215.  

Roughly this area: https://www.openstreetmap.org/#map=12/40.6856/-111.8770 

The overall process for this project was to download the xml file from Open Street Map, saved as 215S.xml.  The file was then renamed to 215S.osm for use in this project.

In this area, and Salt Lake City in general, a common street naming convention is to use the ordinal direction from the center of the city.  Many streets do not have a name, they are called "2700 South", "1300 East", etc.  I expect to find many of these streets to use abberviations like "2700 S".  

I will build a dictionary of street names and then a translation to correct any abbreviations to the full version of the street name.  I expect to find the standard types of street names also, Ave -> Avenue, Ln -> Lane, etc.  Once all the street names are corrected, the data will be exported into csv files to be imported into a MySQL database hosted locally on my PC.  The data will be analyzed using SQL queries.

### Step 1
Parse file and count tags.  The osm file is imported, and parsed through to list the tags and count the elements per tag.

In [1]:
# Define the file to be used for analysis
osmfile = "215s.osm"

# Import python file with tag and key count functions
import parse_count
    
# Call function to count tags
parse_count.count_tags(osmfile)

{'note': 1,
 'meta': 1,
 'bounds': 1,
 'node': 330194,
 'tag': 216705,
 'nd': 386282,
 'way': 47187,
 'member': 13906,
 'relation': 518,
 'osm': 1}

### Step 2
Check k tags.  The k tags are checked for colons and problem characters.  Problem characters will make the data harder to analyze and may need to be cleaned.

In [2]:
# Call function to count keys
parse_count.key_count(osmfile)

{'lower': 88018, 'lower_colon': 113926, 'problemchars': 0, 'other': 14761}

### Define Audit function

First ran audit with just values of Street, Avenue, and Drive.  Added values based on data as valid street types.  Salt Lake City has a lot of streets that do not have names, that just have a value based on the grid system.  Many streets were found with cardinal directions, such as 2700 South, 1300 East, etc.  These will also be found a lot in values to clean N becomes North, etc.

In [3]:
# Import audit function
import audit

# Call audit function.
audit.audit_file(osmfile)

defaultdict(set,
            {'S': {'2100 S',
              '2300 S',
              '3300 S',
              '895 E 4500 S',
              'E 2100 S',
              'E 3300 S',
              'W 2100 S'},
             'Rd': {'Redwood Rd', 'S Redwood Rd'},
             'Dr': {'Atherton Dr',
              'Executive Park Dr',
              'Highland Dr',
              'Ironwood Dr',
              'Millrock Dr',
              'S Highland Dr',
              'W Levoy Dr'},
             'E': {'4400 S 700 E', '4670 S 2300 E'},
             'street': {'State street'},
             'St': {'S State St'},
             '100': {'Myrtle Ave Ste 100', 'West Parkway Boulevard Suite 100'},
             'W.': {'S. 400 W.'},
             'Ave': {'1215 East Wilmington Ave'},
             'Cir': {'Quail Grove Cir'},
             '84119': {'3540 2200 W\nSalt Lake City, UT 84119'},
             '102': {'E 3900 S, #102'},
             's': {'East Downington Ave s'},
             '2100S': {'2100S'},
            

### Mappings

Based on results of audit values are created in a mapping dictionary to correct for abbreviated values.

In [4]:
import update
                
# pass the osm file through the function to correct the street types
update.fix_street(osmfile)

2300 S => 2300 South
W 2100 S => W 2100 South
E 2100 S => E 2100 South
895 E 4500 S => 895 E 4500 South
E 3300 S => E 3300 South
3300 S => 3300 South
2100 S => 2100 South
Redwood Rd => Redwood Road
S Redwood Rd => S Redwood Road
Executive Park Dr => Executive Park Drive
Ironwood Dr => Ironwood Drive
S Highland Dr => S Highland Drive
W Levoy Dr => W Levoy Drive
Millrock Dr => Millrock Drive
Highland Dr => Highland Drive
Atherton Dr => Atherton Drive
4400 S 700 E => 4400 S 700 East
4670 S 2300 E => 4670 S 2300 East
S State St => S Streetate Street
S. 400 W. => S. 400 West
1215 East Wilmington Ave => 1215 East Wilmington Avenue
Quail Grove Cir => Quail Grove Circle
East Downington Ave s => EaSoutht Downington Ave South
Constitution Blvd => Constitution Boulevard
S Decker Lake Blvd => S Decker Lake Boulevard
Nightjar Ln => Nightjar Lane
2700 W => 2700 West
Gazebo Pl => Gazebo Place


### Export to csv files

With the data imported, and the stree names fixed, the data is now exported to csv files that will be imported into a MySQL database.

Functions are found in export.py file.  

schema.py defines the schema used for the files that will be duplicated in the database.

In [5]:
import export
                    
export.process_map(osmfile)

## Analysis

### MySQL database set up
I installed MySQL server on my local machine.  I ran in compatibility issues with importing the data.  I also installed MySQL Workbench and used that for running the SQL.  

I created a schema called OSM in my database, and created the 5 tables to match the csv export.  See CreateTables.sql for the SQL that was used to create the tables.  I also created a quick DropTables.sql script for dropping the tables becasue I ended up changeing the schema details a few times.  For example, I initially had all the id values as int datatype, but the values from my export contained some records with an out of range value, so I changed them all to bigint.  I also had problems with using a column name "key" as a protected term, I updated the tables to use "key2" instead.


### Importing the data
The built in import wizard for MySQL workbench was very slow, pacing to take more than a day to import all the records from the csv files.  I tried to use the infile command in MySQL Workbench, but ran into permission issues.  A security feature of MySQL 8 does not allow this command.  I tried to import the data using Toad for MySQL, but ran into security issues as MySQL does not allow cached passwords for version 8, and Toad could not connect.  I had to uninstall MySQL Server 8 and locate MySQL Server 5.7 installation file.  Once I downgraged to MySQL 5.7, and created the schema and tables again, I was able to import the data using Toad.

### Summary of imported data
The size of the csv files created were:
* nodes.csv - 29 MB
* nodes_tags.csv - 670 KB
* ways.csv = 2.9 MB
* ways_nodes.csv - 9 MB
* ways_tags.csv - 6.6 MB


The record counts for the data once imported were:
* SELECT Count(*) FROM nodes;
    * 330194
* SELECT Count(*) FROM nodes_tags;
    * 18090
* SELECT Count(*) FROM ways;
    * 47187
* SELECT Count(*) FROM ways_nodes;
    * 386282
* SELECT Count(*) FROM ways_tags;
    * 184320

### High level analysis of data

I ran this query to check some user statistics:
```
SELECT user, sum(qty) as total 
from (
SELECT user, count(*) as qty from nodes group by user
union all
select user, count(*) as qty from ways group by user
) as sub 
group by user 
order by total desc, user asc
```

I got back a list of 558 unique user names for edits.  The top 5 users were are listed below.  I also made a change myself in Open Street Map before exporting the data.  My edit is found in the data, under username 'quarlow'.  My edit created a total of 5 edits across the ways and nodes tables.  I added a local park that is new and not yet included in the data.  I combined the users from the nodes table and the ways table to get a complete list of users in the dataset.

| User | Total |
|------|-------|
|UtahBuildingsImport|99517|
|chadbunn|80802|
|osmjwh|72063|
|mash84121|12153|
|mvexel|11664|
|...|...|
|quarlow|5|


### Church Types
I took a look at what denomination of churches have been input into the data for this area.  Salt Lake City is a predominantly Mormon area.  I wanted to see how skewed the records for churches would be to support or contradict that observation.  I used this SQL:

```
SELECT value as Religion, count(*) as total
FROM osm.nodes_tags
WHERE key2 = 'denomination'
GROUP BY value;
```

|Religion|Total|
|--|--|
|"latter_day_saints|1|
|baptist|6|
|catholic|2|
|foursquare|1|
|jehovahs_witness|1|
|lutheran|3|
|mormon|75|
|presbyterian|3|

The Mormon religion's official name is The Church of Jesus Christ of Latter Day Saints.  The record for "latter_day_saints is also another method of identifying as Mormon.  I will also exclude the foursquare record, and that may be an error.  The updated numbers are:

|Religion|Total|
|--|--|
|baptist|6|
|catholic|2|
|jehovahs_witness|1|
|lutheran|3|
|mormon|76|
|presbyterian|3|

There are 91 total records that indicate a denomination, of those 76 are Mormon, or 84%.  This supports the observation of a predominantly Mormon population in this area.

### Possible improvements of data

The religion data provided two records that appear suspect.  This would be one possible way to improve the data, to update the latter_day_saint data to mormon, and to remove the foursquare record.  The impact of this change would be small, as it's only 2 records.  But applied to a larger area of Salt Lake City, or the Wastach Front would have positive impact.  The method for fixing this would be similar to the street type example earlier. A dicitonary of observed values would be created and a mapping used to correct and update any inconsistent values.  Religion can be a sensitive topic, and changing the values to something else could offend the users that imput the data.  Making modifications to religion data may provide cleaner summaries, but by changing the identified religion it may misrepresent how the population actually identifies.